In [6]:
import os
from dotenv import load_dotenv, find_dotenv
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

_ = load_dotenv(find_dotenv())

model = ChatGroq(model = "llama-3.1-8b-instant")

prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")

output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"soccer_player": "Ronaldo"})

"Here's a curious fact about Cristiano Ronaldo: \n\nCristiano Ronaldo has a large collection of cars and is known to be a car enthusiast. However, what's lesser-known is that Ronaldo has a rare genetic condition known as 'port wine stain.' This birthmark is typically found on the face and can sometimes be a sign of Sturge-Weber syndrome, a rare disorder that affects the skin, brain, and eyes.\n\nRonaldo's birthmark covers a significant portion of his face and head, which was noticeable as a young boy. The condition didn't seem to affect his sports career, and he has become a global celebrity.\n\nThis is just one of the many fascinating facts about Cristiano Ronaldo that goes beyond his impressive football career."

## Use of .bind() to add arguments to a Runnable in a LCEL Chain
* For example, we can add an argument to stop the model response when it reaches the word "Ronaldo":

In [3]:
chain_2 = prompt | model.bind(stop=["Ronaldo"]) | output_parser
chain_2.invoke({"soccer_player": "Ronaldo"})

"Here's a curious fact about Cristiano "

# Combining LCEL Chains

In [7]:
second_prompt = ChatPromptTemplate.from_template("Was {soccer_player} authentic?")

composed_chain = {"soccer_player": chain} | second_prompt | model | StrOutputParser()

composed_chain.invoke({"soccer_player": "Ronaldo"})

"Unfortunately, I couldn't find any reliable sources confirming Cristiano Ronaldo obtaining a pilot's license in 2016 or any other year. It is possible that he may be a licensed pilot, but this information is not verified.\n\nHowever, Cristiano Ronaldo does own and fly a private jet, a Bombardier Challenger 350, as reported by multiple sources. He has been spotted flying his private jet on several occasions."

## Another example: a chain inside another chain

In [9]:
from operator import itemgetter

prompt1 = ChatPromptTemplate.from_template("what is the country {politician} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what continent is the country {country} in? respond in {language}"
)

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"country": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

chain2.invoke({"politician": "Miterrand", "language": "english"})

'François Mitterrand was the President of France, serving from 1981 to 1995. The continent where France is located is Europe.'

# LCEL chain at work in a typical RAG app

In [ ]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("What is Task Decomposition?")